<table>
    <tr>
        <td><img src="./imagenes/Macc.png" width="400"/></td>
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp>
                <h1 style="color:blue;text-align:center">Inteligencia Artificial</h1
            </tp>
            <tp>
                <p style="font-size:150%;text-align:center">Markov Decision Processes</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivo <a class="anchor" id="inicio"></a>

En este notebook veremos una manera de implementar los ambientes de tarea de los MDP. Implementaremos manualmente algunos MDP.

Este notebook está basado en las presentación de Sanghi (2021), capítulo 2 y sus [notebooks](https://github.com/Apress/deep-reinforcement-learning-python); Sutton R., & Barto, A., (2015), capítulos 3 y 4; y también Winder, P., (2021), capítulo 2 y su [notebook](https://rl-book.com/learn/mdp/code_driven_intro/). 

# Secciones

Desarrollaremos la explicación de la siguiente manera:

1. [Ejemplos de implementación](#impl).
    1. [GridWorld](#gw).
    2. [Tienda](#tienda).
2. [Evaluación de políticas](#poli-eval).
3. [Mejoramiento de políticas](#dp).
    1. [Policy iteration](#poli-iter).
    2. [Value iteration](#value-iter).
    3. [Comparación de tiempos](#comp).


# Dependencias

Las librerías que usaremos en este notebook son las siguientes. Por favor corra esta celda siempre que inicie el notebook o cuando reinicie el Kernel, instalando las librerías que hagan falta en su ambiente de python: 

In [1]:
import random
import copy
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, OffsetImage, TextArea
from time import sleep
from IPython.display import clear_output
from typing import Tuple
from Tiempos import *

---

# Ejemplos de implementación <a class="anchor" id="impl"></a>

([Volver al inicio](#inicio))

Comenzaremos con la implementación de un ejemplo muy usado para visualizar valores de estados y políticas, que es el ejemplo del Grid World. También tendremos un ejemplo más sencillo, que se le pedirá a usted terminar, que es el de la tienda con un solo artículo, el cual discutimos en las diapositivas de clase. 


## Grid World <a class="anchor" id="gw"></a>

([Volver a Ejemplos](#impl))

Un ejemplo muy útil en términos de visualización de MDP es el del Grid World, el cual consiste de una rejilla rectangular. Las casillas corresponden a los estados. Hay cuatro acciones: norte, sur, este y oeste, que hacen que el agente se mueva una casilla en la dirección respectiva en la rejilla. Las acciones que sacarían al agente de la rejilla dejan su ubicación sin cambios. Cada acción da como resultado una recompensa de -1. Las casillas (0,0) y (4,4) son estados terminales.

Veamos una implementación *ad hoc* tomada del libro de Sanghi (ver [código](https://github.com/Apress/deep-reinforcement-learning-python/blob/main/chapter3/gridworld.py)):

In [2]:
# define the actions
NORTH = 0
EAST = 1
SOUTH = 2
WEST = 3
dict_acciones = {0:"⬆", 1:"➡", 2:"⬇", 3:"⬅"}

class GridworldEnv():
    """
    A 4x4 Grid World environment from Sutton's Reinforcement 
    Learning book chapter 4. Termial states are top left and
    the bottom right corner.
    Actions are (UP=0, RIGHT=1, DOWN=2, LEFT=3).
    Actions going off the edge leave agent in current state.
    Reward of -1 at each step until agent reachs a terminal state.
    """

    def __init__(self, shape=(4,4)):
        self.shape = shape
        self.nS = np.prod(self.shape)
        self.nA = 4
        self.action_space = list(range(self.nA))
        self.state = 2
        P = {}
        for s in range(self.nS):
            P[s] = {a: [] for a in range(self.nA)}
            # Per state and action provide list as follows
            # P[state][action] = [(probability, next_state, reward, done)]
            # Assignment is obtained by means of method _transition_prob
            position = np.unravel_index(s, self.shape)
            P[s][NORTH] = self._transition_prob(position, [-1, 0])
            P[s][EAST] = self._transition_prob(position, [0, 1])
            P[s][SOUTH] = self._transition_prob(position, [1, 0])
            P[s][WEST] = self._transition_prob(position, [0, -1])
        # We expose the model of the environment for dynamic programming
        # This should not be used in any model-free learning algorithm
        self.P = P

    def _limit_coordinates(self, coord):
        """
        Prevent the agent from falling out of the grid world
        :param coord:
        :return:
        """
        coord[0] = min(coord[0], self.shape[0] - 1)
        coord[0] = max(coord[0], 0)
        coord[1] = min(coord[1], self.shape[1] - 1)
        coord[1] = max(coord[1], 0)
        return coord

    def _transition_prob(self, current, delta):
        """
        Model Transitions. Prob is always 1.0.
        :param current: Current position on the grid as (row, col)
        :param delta: Change in position for transition
        :return: [(1.0, new_state, reward, done)]
        """
        # if stuck in terminal state
        current_state = np.ravel_multi_index(tuple(current), self.shape)
        if current_state == 0 or current_state == self.nS - 1:
            return [(1.0, current_state, 0, True)]
        new_position = np.array(current) + np.array(delta)
        new_position = self._limit_coordinates(new_position).astype(int)
        new_state = np.ravel_multi_index(tuple(new_position), self.shape)
        is_done = new_state == 0 or new_state == self.nS - 1
        return [(1.0, new_state, -1, is_done)]

    def reset(self):
        self.state = random.randint(1, self.nS - 2)
        return self.state
    
    def step(self, action):
        s = self.state
        p = self.P[s][action]
        indice = random.choices(
            population = range(len(p)),
            weights = [x[0] for x in p],
            k = 1
        )[0]
        new_state = p[indice][1]
        self.state = new_state
        reward = p[indice][2]
        done = p[indice][3]
        return new_state, reward, done    

    def render(self):
        state = self.state
        output = ''
        for s in range(self.nS):
            if s == state:
                if state == 0 or state == self.nS - 1:
                    output += '@'
                else:
                    output += "x"
            # Print terminal state
            elif s == 0 or s == self.nS - 1:
                output += "o"
            else:
                output += "_"
            position = np.unravel_index(s, self.shape)
            if position[1] == 0:
                output = output.lstrip()
            if position[1] == self.shape[1] - 1:
                output = output.rstrip()
                output += '\n'
        print(output)

    def __str__(self):
        string = ''
        for s in range(self.nS):
            string += '\n'+'-'*20
            string += f'\nState:{np.unravel_index(s, self.shape)}'
            for a in range(self.nA):
                string += f'\nAction:{dict_acciones[a]}'
                for x in self.P[s][a]:
                    string += f'\n| probability:{x[0]}, '
                    string += f'new_state:{np.unravel_index(x[1], self.shape)}, '
                    string += f'reward:{x[2]}, '
                    string += f'done?:{x[3]} |'
        return string

Observemos algunas características de la clase `GridworldEnv`:

In [3]:
env = GridworldEnv()
print("Acciones posibles:", env.action_space)
print('')
print("Número de acciones posibles:", len(env.action_space))
print('')
a = random.choice(env.action_space)
print("Una acción posible seleccionada al azar:", dict_acciones[a])
print('')
print("El estado actual es:", env.state)

Acciones posibles: [0, 1, 2, 3]

Número de acciones posibles: 4

Una acción posible seleccionada al azar: ➡

El estado actual es: 2


Al imprimir el objeto, podemos ver el modelo del MDP que ha sido implementado mediante el método `_transition_prob`, el cual define, para cada estado y acción, la siguiente tupla: 

(probabilidad, próximo estado, recompensa, finalizado)

In [4]:
print(env)


--------------------
State:(0, 0)
Action:⬆
| probability:1.0, new_state:(0, 0), reward:0, done?:True |
Action:➡
| probability:1.0, new_state:(0, 0), reward:0, done?:True |
Action:⬇
| probability:1.0, new_state:(0, 0), reward:0, done?:True |
Action:⬅
| probability:1.0, new_state:(0, 0), reward:0, done?:True |
--------------------
State:(0, 1)
Action:⬆
| probability:1.0, new_state:(0, 1), reward:-1, done?:False |
Action:➡
| probability:1.0, new_state:(0, 2), reward:-1, done?:False |
Action:⬇
| probability:1.0, new_state:(1, 1), reward:-1, done?:False |
Action:⬅
| probability:1.0, new_state:(0, 0), reward:-1, done?:True |
--------------------
State:(0, 2)
Action:⬆
| probability:1.0, new_state:(0, 2), reward:-1, done?:False |
Action:➡
| probability:1.0, new_state:(0, 3), reward:-1, done?:False |
Action:⬇
| probability:1.0, new_state:(1, 2), reward:-1, done?:False |
Action:⬅
| probability:1.0, new_state:(0, 1), reward:-1, done?:False |
--------------------
State:(0, 3)
Action:⬆
| probabili

Uno de los métodos más importantes de la clase es el `step()`, el cual recibe una acción como argumento y, junto con la información del estado actual y el modelo de transiciones, obtiene el estado al que pasa el sistema y devuelve una recompensa. También se obtiene un valor booleano que indica si el estado obtenido es terminal o no:

In [6]:
env = GridworldEnv()
obs, reward, done = env.step(SOUTH)
print(f'Estado={obs}, Recompensa={reward}, Finalizado={done}')
env.render()

Estado=6, Recompensa=-1, Finalizado=False
o___
__x_
____
___o



<a class="anchor" id="ej1"></a>**Ejercicio 1:** 

([Próximo ejercicio](#ej2))

Cree una pequeña función para hacer una caminata aleatoria por la rejilla hasta que el agente llegue a un estado terminal. Encuentre la utlidad del episodio.

In [9]:
env = GridworldEnv()
env.render()
done = False
utilidad = 0

while not done:
    
    action = random.choice(env.action_space)
    obs, reward, done = env.step(action)
    utilidad += reward
    clear_output(wait = True)
    env.render()
    sleep(.10)

print(utilidad)

@___
____
____
___o

-6


---

## Tienda <a class="anchor" id="tienda"></a>

([Volver a Ejemplos](#impl))

Recordemos el ejemplo discutido en las diapositivas. Imagine que estamos a cargo de una tienda que solo vende un artículo. Es necesario tener existencias del mismo para poder venderlas, pero como el local lo cobran por metro cuadrado, no queremos tener demasiadas existencias. 

<img src="./imagenes/tienda.png" width="200"/>

La implementación inicial es la siguiente:

In [18]:
# define the actions
NONE = 0
RESTOCK = 1
dict_acciones = {0:'None', 1:'RESTOCK'}

class TiendaEnv():

    def __init__(self, p_sale=0.2):
        self.nS = 3 # number of states
        self.nA = 2 # number of actions
        self.p_sale = p_sale # probability of selling
        self.state = 2

        P = {} # initialize model (dict of dicts)
        for s in range(self.nS):
            P[s] = {a: [] for a in range(self.nA)}
            # Per state and action provide list as follows
            # P[state][action] = [(probability, next_state, reward, done)]
            # Assignment is obtained by means of method _transition_prob
            P[s][NONE] = self._transition_prob(s, NONE)
            if s != 2:
                P[s][RESTOCK] = self._transition_prob(s, RESTOCK)
        self.P = P
        
    def _transition_prob(self, state, action):
        if state == 0 and action == NONE:
            return [(1, 0, 0, False)]
        else:
            if action == NONE:
                return [(self.p_sale, state-1, 1, False), (1-self.p_sale, state, 0, False)]
            else:
                return [(self.p_sale, state, 1, False), (1-self.p_sale, state+1, 0, False)]            
       
    def render(self):
        fig, axes = plt.subplots(figsize=(3, 1))
        imagen = "./imagenes/caja.png"
        arr_img = plt.imread(imagen, format='png')
        imagen = OffsetImage(arr_img, zoom=0.5)
        imagen.image.axes = axes
        paso = 1./3
        offsetX = 0
        offsetY = 0
        Y = 0
        for X in range(0, self.state):
            ab = AnnotationBbox(
                    imagen,
                    [(X*paso) + offsetX, (Y*paso) + offsetY],
                    frameon=False)
            axes.add_artist(ab)
        axes.axis('off')
        plt.show()

    def __str__(self):
        string = ''
        for s in range(self.nS):
            string += '\n'+'-'*20
            string += f'\nState:{s}'
            for a in range(self.nA):
                string += f'\nAction:{dict_acciones[a]} | {self.P[s][a]} |'
        return string
    
    def step(self, action):
        pass
        # AQUÍ COMIENZA SU CÓDIGO
        
        s = self.state
        p = self.P[s][action]
        
        indice = random.choices(
            population = range(len(p)),
            weights = [x[0] for x in p],
            k = 1)[0]
        
        new_state = p[indice][1]
        self.state = new_state
        reward = p[indice][2]
        done = p[indice][3]
        return new_state, reward, done 
        
        # AQUÍ TERMINA SU CÓDIGO
        
    def get_valid_actions():
        pass
        # AQUÍ COMIENZA SU CÓDIGO
        
        # AQUÍ TERMINA SU CÓDIGO


In [19]:
# create environment
env = TiendaEnv(p_sale=0.4)

Podemos ver el modelo del MDP que ha sido implementado mediante el método `_transition_prob`, el cual define, para cada estado y acción, la tupla 

(probabilidad, próximo estado, recompensa, finalizado)

In [20]:
print(env)


--------------------
State:0
Action:None | [(1, 0, 0, False)] |
Action:RESTOCK | [(0.4, 0, 1, False), (0.6, 1, 0, False)] |
--------------------
State:1
Action:None | [(0.4, 0, 1, False), (0.6, 1, 0, False)] |
Action:RESTOCK | [(0.4, 1, 1, False), (0.6, 2, 0, False)] |
--------------------
State:2
Action:None | [(0.4, 1, 1, False), (0.6, 2, 0, False)] |
Action:RESTOCK | [] |


Por ejemplo, vemos que con el estado 0 y la acción 0 (NONE) se tiene

(1, 0, 0, False)

Esto quiere decir que con probabilidad 1 el estado que se obtiene es el estado 0, con recompensa 0. Además, el estado 0 no es estado final.

<a class="anchor" id="ej2"></a>**Ejercicio 2:** 

([Anterior ejercicio](#ej1)) ([Próximo ejercicio](#ej3))

Uno de los métodos más importantes que no está implementado es el de `step()`, el cual recibe una acción como argumento y, junto con la información del estado actual y el modelo de transiciones, obtiene una tripla `(state, reward, done)`, que consiste del estado al que pasa el sistema, la recompensa obtenida, y un valor booleano que indica si el estado obtenido es terminal o no. Implemente dicho método en la clase `TiendaEnv`. 

**Nota:** Observe que el modelo está guardado en el atributo `self.P` de la forma 

P[state][action] = [(probability, next_state, reward, done)]


In [21]:
#def step(self, action):
#    pass
    # AQUÍ SU CÓDIGO 
    
    # AQUÍ TERMINA SU CÓDIGO

#setattr(TiendaEnv, "step", step)

env = TiendaEnv(p_sale=0.4)
print('Estado inicial:', env.state)
a = 0
obs, reward, done = env.step(a)
print(f'Estado obtenido con la acción {a}: {obs}')
print(f'Recompensa obtenida con la acción {a}: {reward}')
print('Finalizado?', done)

Estado inicial: 2
Estado obtenido con la acción 0: 2
Recompensa obtenida con la acción 0: 0
Finalizado? False


---

<a class="anchor" id="ej3"></a>**Ejercicio 3:** 

([Anterior ejercicio](#ej2)) ([Próximo ejercicio](#ej4))


Otro método importante que no está implementado es el de determinar cuáles acciones son posibles en un estado dado, llamémoslo `get_valid_actions()`. Implemente dicho método en la clase `TiendaEnv`. 

Observe que la acción NONE es posible en todos los estados, pero la acción RESTOCK no es posible en el estado 2. 

---

# Evaluación de políticas <a class="anchor" id="poli-eval"></a>

([Volver al inicio](#inicio))

Vamos a usar la programación dinámica para encontrar los valores de estado para una política dada. La idea central es usar la ecuación de Bellman como una regla iterativa:

$$v_{k+1}(s) = \sum_{s'}\left( p(s' | s,\pi(s)) \Bigl[ r + \gamma v_k(s') \Bigr] \right)$$

Esto da lugar al siguiente algoritmo:

<img src="./imagenes/policy_evaluation.png" width="auto"/>

<a class="anchor" id="ej4"></a>**Ejercicio 4:** 

([Anterior ejercicio](#ej3)) ([Próximo ejercicio](#ej5))

Implemente el algoritmo iterativo de evaluación de políticas y utilícelo para encontrar los valores de la política `policy` definida en la siguiente celda:

In [ ]:
dict_acciones = {0:"⬆", 1:"➡", 2:"⬇", 3:"⬅"}
policy = [NORTH, EAST, EAST, SOUTH] * 4
pp = np.reshape(policy, (4,4))
print(np.vectorize(dict_acciones.get)(pp))

In [ ]:
# Policy evaluation

def policy_eval(env, policy, discount_factor=1.0, theta=0.01, verbose=0):
    """
    Evalúa una política para un entorno.
    Input:
        - env: transition dynamics of the environment.
            env.P[s][a] [(prob, next_state, reward, done)].
            env.nS is number of states in the environment.
            env.nA is number of actions in the environment.
        - policy: vector de longitud env.nS que representa la política
        - discount_factor: Gamma discount factor.
        - theta: Stop iteration once value function change is
            less than theta for all states.
        - verbose: 0 no imprime nada, 
                   1 imprime la iteración del valor
    Output:
        Vector de longitud env.nS que representa la función de valor.
    """
    pass
    # AQUÍ SU CÓDIGO
    
    # AQUÍ TERMINA SU CÓDIGO

shape = (4,4)
env = GridworldEnv()
policy = [NORTH, EAST, EAST, SOUTH] * 4
p = policy_eval(env, policy, discount_factor=1, theta=0.1, verbose=0)
pp = np.reshape(p, shape)
print(pp)

**Nota:** El resultado de debe ser 
```
[[ 0. -5. -4. -3.]
 [-1. -4. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
```

---

# Mejoramiento de políticas <a class="anchor" id="dp"></a>

([Volver al inicio](#inicio))

Recordemos que el propósito del RL es encontrar la acción que mejor utilidad tenga en cada estado, determinando así la política óptima para el problema. Si conocemos el modelo de un MDP, podemos ir mejorando una política paso a paso. Los dos métodos de esta sección realizan el mejoramiento de una política hasta llegar a la política óptima. 

## Policy iteration <a class="anchor" id="poli-iter"></a>

([Volver a Mejoramiento](#dp))

En este algoritmo se busca mejorar una política $\pi$ en cada estado $s$, definiendo $\pi'$ de tal manera que:

$$\pi'(s) = \mbox{arg}\max_a q_{\pi}(s,a)$$

Esto da lugar a una nueva política $\pi'$. Luego, se recalculan los valores $v_{\pi'}(s)$ usando el algoritmo de evaluación de política visto en la sección anterior. Este proceso se itera hasta converger a la política óptima:

<img src="./imagenes/p_i.png" width="350"/>

El algoritmo es el siguiente:

<img src="./imagenes/policy_iteration1.png" width="auto"/>

<a class="anchor" id="ej5"></a>**Ejercicio 5:** 

([Anterior ejercicio](#ej4)) ([Próximo ejercicio](#ej6))

Implemente el algoritmo de policy improvement y mejore la política del ejercicio 4.

In [ ]:
# Policy Improvement

def policy_iteration(env, pol, discount_factor=1.0, theta=0.01, verbose=0):      
    """
    Mejoramiento de una política.
    Input:
        - env: OpenAI env. env.P -> transition dynamics of the environment.
            env.P[s][a] [(prob, next_state, reward, done)].
            env.nS is number of states in the environment.
            env.nA is number of actions in the environment.
        - pol: vector de longitud env.nS que representa la política
        - discount_factor: Gamma discount factor.
        - theta: Stop iteration once value function change is
            less than theta for all states.
        - verbose: 0 no imprime nada, 
                   1 imprime la iteración de la política,
                   2 imprime también la iteración del valor
    Output:
        Vector de longitud env.nS que representa la política óptima.
    """    
    pass
    # AQUÍ SU CÓDIGO
    
    # AQUÍ TERMINA SU CÓDIGO

shape = (4,4)
env = GridworldEnv()
policy = [NORTH, EAST, EAST, SOUTH] * 4
pp = np.reshape(policy, shape)
print('Política inicial:')
print(np.vectorize(dict_acciones.get)(pp))
print('')
p = policy_iteration(env, policy, discount_factor=1, theta=0.01, verbose=0)
pp = np.reshape(p, shape)
print('Política óptima:')
print(np.vectorize(dict_acciones.get)(pp))

**Nota:** La respuesta debe ser:

```
Política óptima:
[['⬆' '⬅' '⬅' '⬇']
 ['⬆' '⬆' '⬆' '⬇']
 ['⬆' '⬆' '➡' '⬇']
 ['⬆' '➡' '➡' '⬆']]
```

---

## Value iteration <a class="anchor" id="value-iter"></a>

([Volver a Mejoramiento](#dp))

Para mejorar el desempeño del algoritmo de policy iteration, se puede truncar la evaluación de la política después de una iteración para cada estado. Además, se puede combinar, en una sola regla iterativa, el mejoramiento de la política con la evaluación truncada de la política:

$$v_{k+1}(s) = \max_{a}\sum_{s'}\left( p(s' | s, a) \Bigl[ r + \gamma  v_{k}(s') \Bigr] \right)$$

Se puede demostrar que la sucesión $\{v_k\}$ converge a $v_*$. 

Finalmente, para obtener la política óptima $\pi_*$ se buscan las acciones mediante el argmax de los valores óptimos obtenidos en el proceso anterior:

$$\pi_*(s) = \mbox{arg}\max_a\sum_{s'}\left( p(s' | s, a) \Bigl[ r + \gamma  v_{*}(s') \Bigr] \right)$$

El algoritmo es el siguiente:

<img src="./imagenes/value_iteration.png" width="auto"/>

<a class="anchor" id="ej6"></a>**Ejercicio 6:** 

([Anterior ejercicio](#ej5))

Implemente el algoritmo de value-iteration para encontrar la política óptima del MDP. Use su algoritmo para encontrar la política óptima del Grid World.

In [ ]:
# Value iteration

def value_iteration(env, discount_factor=1.0, theta=0.01, verbose=0):
    """
    Mejoramiento de una política.
    Input:
        - env: OpenAI env. env.P -> transition dynamics of the environment.
            env.P[s][a] [(prob, next_state, reward, done)].
            env.nS is number of states in the environment.
            env.nA is number of actions in the environment.
        - discount_factor: Gamma discount factor.
        - theta: Stop iteration once value function change is
            less than theta for all states.
        - verbose: 0 no imprime nada, 
                   1 imprime la iteración de la política,
                   2 imprime también la iteración del valor
    Output:
        Vector de longitud env.nS que representa la política óptima.
    """ 
    pass
    # AQUÍ SU CÓDIGO
    
    # AQUÍ TERMINA SU CÓDIGO

shape = (4,4)
env = GridworldEnv()
p = value_iteration(env, discount_factor=1, theta=0.01, verbose=0)
pp = np.reshape(p, shape)
print('Política óptima:')
print(np.vectorize(dict_acciones.get)(pp))

---

## Comparación de tiempos <a class="anchor" id="comp"></a>

([Volver a Mejoramiento](#dp))

Vamos a hacer el estudio empírico de la complejidad de tiempos de los dos algoritmos. Correremos ambos algoritmos sobre el Grid World con tamaños (4,4) hasta (9,9). Con cada ambiente correremos 10 cada algoritmo y registraremos los tiempos de máquina usados para encontrar la política óptima. Los resultados son los siguientes: 

In [ ]:
p_i = lambda env: policy_iteration(env, get_nice_policy(env.shape))
v_i = lambda env: value_iteration(env)
funs = [p_i, v_i]
nombres_funs = ['policy-iteration', 'value-iteration']
shapes = [(n,n) for n in range(4,15)]
lista_args = [GridworldEnv(shape) for shape in shapes]
df = compara_entradas_funs(funs, nombres_funs, lista_args, N=10)
sns.lineplot(x='Long_entrada',y='Tiempo',hue='Funcion',data=df)

La gráfica muestra el tiempo promedio que cada algoritmo toma para encontrar la política óptima con distintos tamaños del Grid World. A partir de la gráfica queda muy claro que el algoritmo de `value-iteration` es más eficiente que el de `policy-iteration`. Observe también que el primero no necesita una política de entrada, mientras que sí se requiere una política como argumento del `policy-iteration`. En este ejemplo se tomó una política que converge relativamente rápido, pero otras políticas toman muchísimo más tiempo en converger. Todo esto muestra las ventajas del `value-iteration`, el cual es más rápido y no requiere política de entrada.

Adicionalmente, observe que la complejidad de tiempo va creciendo mucho en ambos casos, lo cual hace que sean inviables para MDPs que tienen una gran cantidad de estados.

# En este notebook usted aprendió

* Cómo implementar MDP en python usando la librería `gym` de OpenAI.
* Cómo implementar la evaluación de una política, para obtener los valores en cada estado.
* Cómo implementar la metodología de mejoramiento de políticas mediante policy iteration y value iteration.

# Bibliografía

([Volver al inicio](#inicio))

Shanghi, N. (2021) Deep Reinforcement Learning with Python: With PyTorch, TensorFlow and OpenAI Gym. Apress. 

Sutton R., & Barto, A., (2015) Reinforcement Learning: An Introduction, 2nd Edition. A Bradford Book. Series: Adaptive Computation and Machine Learning series. 

Winder, P., (2021) Reinforcement Learning: Industrial Applications of Intelligent Agents. O’Relly.